# Project 6: Analyzing Stock Sentiment from Twits
## Instructions
Each problem consists of a function to implement and instructions on how to implement the function.  The parts of the function that need to be implemented are marked with a `# TODO` comment.

## Packages
When you implement the functions, you'll only need to you use the packages you've used in the classroom, like [Pandas](https://pandas.pydata.org/) and [Numpy](http://www.numpy.org/). These packages will be imported for you. We recommend you don't add any import statements, otherwise the grader might not be able to run your code.

### Load Packages

In [1]:
import json
import nltk
import os
import random
import re
import torch
import numpy as np

from torch import nn, optim
import torch.nn.functional as F

## Introduction
When deciding the value of a company, it's important to follow the news. For example, a product recall or natural disaster in a company's product chain. You want to be able to turn this information into a signal. Currently, the best tool for the job is a Neural Network. 

For this project, you'll use posts from the social media site [StockTwits](https://en.wikipedia.org/wiki/StockTwits). The community on StockTwits is full of investors, traders, and entrepreneurs. Each message posted is called a Twit. This is similar to Twitter's version of a post, called a Tweet. You'll build a model around these twits that generate a sentiment score.

We've collected a bunch of twits, then hand labeled the sentiment of each. To capture the degree of sentiment, we'll use a five-point scale: very negative, negative, neutral, positive, very positive. Each twit is labeled -2 to 2 in steps of 1, from very negative to very positive respectively. You'll build a sentiment analysis model that will learn to assign sentiment to twits on its own, using this labeled data.

The first thing we should to do, is load the data.

## Import Twits 
### Load Twits Data 
This JSON file contains a list of objects for each twit in the `'data'` field:

```
{'data':
  {'message_body': 'Neutral twit body text here',
   'sentiment': 0},
  {'message_body': 'Happy twit body text here',
   'sentiment': 1},
   ...
}
```

The fields represent the following:

* `'message_body'`: The text of the twit.
* `'sentiment'`: Sentiment score for the twit, ranges from -2 to 2 in steps of 1, with 0 being neutral.


To see what the data look like by printing the first 10 twits from the list. 

In [2]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'twits.json'), 'r') as f:
    twits = json.load(f)

print(twits['data'][:10])

[{'message_body': '$FITB great buy at 26.00...ill wait', 'sentiment': 2, 'timestamp': '2018-07-01T00:00:09Z'}, {'message_body': '@StockTwits $MSFT', 'sentiment': 1, 'timestamp': '2018-07-01T00:00:42Z'}, {'message_body': '#STAAnalystAlert for $TDG : Jefferies Maintains with a rating of Hold setting target price at USD 350.00. Our own verdict is Buy  http://www.stocktargetadvisor.com/toprating', 'sentiment': 2, 'timestamp': '2018-07-01T00:01:24Z'}, {'message_body': '$AMD I heard there’s a guy who knows someone who thinks somebody knows something - on StockTwits.', 'sentiment': 1, 'timestamp': '2018-07-01T00:01:47Z'}, {'message_body': '$AMD reveal yourself!', 'sentiment': 0, 'timestamp': '2018-07-01T00:02:13Z'}, {'message_body': '$AAPL Why the drop? I warren Buffet taking out his position?', 'sentiment': 1, 'timestamp': '2018-07-01T00:03:10Z'}, {'message_body': '$BA bears have 1 reason on 06-29 to pay more attention https://dividendbot.com?s=BA', 'sentiment': -2, 'timestamp': '2018-07-01T

### Length of Data
Now let's look at the number of twits in dataset. Print the number of twits below.

In [3]:
"""print out the number of twits"""

# TODO Implement 
print(len(twits['data']))


1548010


### Split Message Body and Sentiment Score

In [4]:
messages = [twit['message_body'] for twit in twits['data']]
# Since the sentiment scores are discrete, we'll scale the sentiments to 0 to 4 for use in our network
sentiments = [twit['sentiment'] + 2 for twit in twits['data']]

## Preprocessing the Data
With our data in hand we need to preprocess our text. These twits are collected by filtering on ticker symbols where these are denoted with a leader $ symbol in the twit itself. For example,

`{'message_body': 'RT @google Our annual look at the year in Google blogging (and beyond) http://t.co/sptHOAh8 $GOOG',
 'sentiment': 0}`

The ticker symbols don't provide information on the sentiment, and they are in every twit, so we should remove them. This twit also has the `@google` username, again not providing sentiment information, so we should also remove it. We also see a URL `http://t.co/sptHOAh8`. Let's remove these too.

The easiest way to remove specific words or phrases is with regex using the `re` module. You can sub out specific patterns with a space:

```python
re.sub(pattern, ' ', text)
```
This will substitute a space with anywhere the pattern matches in the text. Later when we tokenize the text, we'll split appropriately on those spaces.

### Pre-Processing

In [5]:
nltk.download('wordnet')


def preprocess(message):
    """
    This function takes a string as input, then performs these operations: 
        - lowercase
        - remove URLs
        - remove ticker symbols 
        - removes punctuation
        - tokenize by splitting the string on whitespace 
        - removes any single character tokens
    
    Parameters
    ----------
        message : The text message to be preprocessed.
        
    Returns
    -------
        tokens: The preprocessed text into tokens.
    """ 
    #TODO: Implement 
    
    # Lowercase the twit message
    text = message.lower()
    
    # Replace URLs with a space in the message
    text = re.sub(r'http:\/\/.*[\r\n]*', ' ', text)
    
    # Replace ticker symbols with a space. The ticker symbols are any stock symbol that starts with $.
    text = re.sub(r'[$][A-za-z][\S]*', ' ', text)
    
    # Replace StockTwits usernames with a space. The usernames are any word that starts with @.
    text = re.sub(r'[@][A-za-z][\S]*', ' ', text)

    # Replace everything not a letter with a space
    text =  re.sub(r'[\W_]', ' ', text)
    
    # Tokenize by splitting the string on whitespace into a list of words
    tokens = text.split()

    # Lemmatize words using the WordNetLemmatizer. You can ignore any word that is not longer than one character.
    wnl = nltk.stem.WordNetLemmatizer()
    tokens = [wnl.lemmatize(t) for t in tokens if len(t) > 1]
    
    return tokens

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


### Preprocess All the Twits 
Now we can preprocess each of the twits in our dataset. Apply the function `preprocess` to all the twit messages.

In [6]:
# TODO Implement

tokenized = [preprocess(m) for m in messages]
tokenized

[['great', 'buy', 'at', '26', '00', 'ill', 'wait'],
 [],
 ['staanalystalert',
  'for',
  'jefferies',
  'maintains',
  'with',
  'rating',
  'of',
  'hold',
  'setting',
  'target',
  'price',
  'at',
  'usd',
  '350',
  '00',
  'our',
  'own',
  'verdict',
  'is',
  'buy'],
 ['heard',
  'there',
  'guy',
  'who',
  'know',
  'someone',
  'who',
  'think',
  'somebody',
  'know',
  'something',
  'on',
  'stocktwits'],
 ['reveal', 'yourself'],
 ['why',
  'the',
  'drop',
  'warren',
  'buffet',
  'taking',
  'out',
  'his',
  'position'],
 ['bear',
  'have',
  'reason',
  'on',
  '06',
  '29',
  'to',
  'pay',
  'more',
  'attention',
  'http',
  'dividendbot',
  'com',
  'ba'],
 ['ok',
  'good',
  'we',
  '39',
  're',
  'not',
  'dropping',
  'in',
  'price',
  'over',
  'the',
  'weekend',
  'lol'],
 ['daily', 'chart', 'we', 'need', 'to', 'get', 'back', 'to', 'above', '50'],
 ['drop',
  'per',
  'week',
  'after',
  'spike',
  'if',
  'no',
  'news',
  'in',
  'month',
  'back',
  '

### Bag of Words
Now with all of our messages tokenized, we want to create a vocabulary and count up how often each word appears in our entire corpus. Use the [`Counter`](https://docs.python.org/3.1/library/collections.html#collections.Counter) function to count up all the tokens.

In [7]:
from collections import Counter


"""
Create a vocabulary by using Bag of words
"""

# TODO: Implement 
bow = Counter(' '.join([' '.join(tokens) for tokens in tokenized]).split())
bow

Counter({'great': 17751,
         'buy': 79263,
         'at': 140397,
         '26': 16918,
         '00': 23688,
         'ill': 890,
         'wait': 11389,
         'staanalystalert': 4072,
         'for': 281187,
         'jefferies': 1199,
         'maintains': 924,
         'with': 71572,
         'rating': 16458,
         'of': 217828,
         'hold': 22635,
         'setting': 3924,
         'target': 18130,
         'price': 40197,
         'usd': 2940,
         '350': 2590,
         'our': 12323,
         'own': 9129,
         'verdict': 3747,
         'is': 288016,
         'heard': 1363,
         'there': 25745,
         'guy': 13454,
         'who': 17999,
         'know': 20330,
         'someone': 5194,
         'think': 24386,
         'somebody': 1100,
         'something': 6047,
         'on': 247485,
         'stocktwits': 35344,
         'reveal': 252,
         'yourself': 1212,
         'why': 23214,
         'the': 406184,
         'drop': 18208,
         'warre

### Frequency of Words Appearing in Message
With our vocabulary, now we'll remove some of the most common words such as 'the', 'and', 'it', etc. These words don't contribute to identifying sentiment and are really common, resulting in a lot of noise in our input. If we can filter these out, then our network should have an easier time learning.

We also want to remove really rare words that show up in a only a few twits. Here you'll want to divide the count of each word by the number of messages. Then remove words that only appear in some small fraction of the messages.

In [8]:
"""
Set the following variables:
    freqs
    low_cutoff
    high_cutoff
    K_most_common
"""

# TODO Implement 
import operator 

# Dictionary that contains the Frequency of words appearing in messages.
# The key is the token and the value is the frequency of that word in the corpus.
freqs = {word: count/len(bow) for word, count in bow.items()}

# Float that is the frequency cutoff. Drop words with a frequency that is lower or equal to this number.
low_cutoff = 1e-4

# Integer that is the cut off for most common words. Drop words that are the `high_cutoff` most common words.
high_cutoff = 50

# The k most common words in the corpus. Use `high_cutoff` as the k.
K_most_common = [w[0] for w in sorted(freqs.items(), key=operator.itemgetter(1))[-high_cutoff:]]


filtered_words = [word for word in freqs if (freqs[word] > low_cutoff and word not in K_most_common)]
print(K_most_common)
len(filtered_words) 

['wa', 'by', 'they', 'more', 'earnings', 'out', 'market', 'now', 'if', 'all', 'report', 'we', 'from', 'day', 'call', 'not', 'down', 'with', 'just', 'here', 'today', 'buy', 'what', 'short', 'that', '10', '2018', 'you', 'www', 'be', 'stock', 'are', 'utm', 'up', 'amp', 'will', 'at', '39', 'it', 'com', 'this', 'in', 'and', 'of', 'http', 'on', 'for', 'is', 'to', 'the']


17601

### Updating Vocabulary by Removing Filtered Words
Let's creat three variables that will help with our vocabulary.

In [9]:
"""
Set the following variables:
    vocab
    id2vocab
    filtered
"""

#TODO Implement
# A dictionary for the `filtered_words`. The key is the word and value is an id that represents the word. 
vocab ={word: index for index, word in enumerate(filtered_words, 1)}
# Reverse of the `vocab` dictionary. The key is word id and value is the word. 
id2vocab = {ii: word for word, ii in vocab.items()}
# tokenized with the words not in `filtered_words` removed.
filtered = [[word for word in message if word in vocab] for message in tokenized]

### Balancing the classes
Let's do a few last pre-processing steps. If we look at how our twits are labeled, we'll find that 50% of them are neutral. This means that our network will be 50% accurate just by guessing 0 every single time. To help our network learn appropriately, we'll want to balance our classes.
That is, make sure each of our different sentiment scores show up roughly as frequently in the data.

What we can do here is go through each of our examples and randomly drop twits with neutral sentiment. What should be the probability we drop these twits if we want to get around 20% neutral twits starting at 50% neutral? We should also take this opportunity to remove messages with length 0.

In [10]:
balanced = {'messages': [], 'sentiments':[]}

n_neutral = sum(1 for each in sentiments if each == 2)
N_examples = len(sentiments)
keep_prob = (N_examples - n_neutral)/4/n_neutral
print(keep_prob, N_examples, n_neutral)
for idx, sentiment in enumerate(sentiments):
    message = filtered[idx]

    if len(message) == 0:
        # skip this message because it has length zero
        continue
    elif sentiment != 2 or random.random() < keep_prob:
        balanced['messages'].append(message)
        balanced['sentiments'].append(sentiment)

0.3016022730997995 1548010 701597


If you did it correctly, you should see the following result 

In [11]:
n_neutral = sum(1 for each in balanced['sentiments'] if each == 2)
N_examples = len(balanced['sentiments'])
n_neutral/N_examples

0.19537554494017254

Finally let's convert our tokens into integer ids which we can pass to the network.

In [12]:
token_ids = [[vocab[word] for word in message] for message in balanced['messages']]
sentiments = balanced['sentiments']

## Neural Network
Now we have our vocabulary which means we can transform our tokens into ids, which are then passed to our network. So, let's define the network now!

Here is a nice diagram showing the network we'd like to build: 

#### Embed -> RNN -> Dense -> Softmax
### Implement the text classifier
Before we build text classifier, if you remember from the other network that you built in  "Sentiment Analysis with an RNN"  exercise  - which there, the network called " SentimentRNN", here we named it "TextClassifer" - consists of three main parts: 1) init function `__init__` 2) forward pass `forward`  3) hidden state `init_hidden`. 

This network is pretty similar to the network you built expect in the  `forward` pass, we use softmax instead of sigmoid. The reason we are not using sigmoid is that the output of NN is not a binary. In our network, sentiment scores have 5 possible outcomes. We are looking for an outcome with the highest probability thus softmax is a better choice.

In [13]:
class TextClassifier(nn.Module):

    """
    Initialize the model by setting up the layers.

        Parameters
        ---------- 
        Use the following parameters as the arguments for __init__ function: 
        (self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1)
        - You can add more layers or change the dropout rate. 
    """
    def __init__(self, vocab_size, embed_size, lstm_size, output_size, lstm_layers=1, dropout=0.1):
        super().__init__()
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.lstm_layers = lstm_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, lstm_size, lstm_layers, dropout=dropout)

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_size, output_size)

    # forward pass 

    """
      Perform a forward pass of our model on some input and hidden state.

        Parameters
        ----------
              Use softmax instead of sigmoid

        Returns
        -------
            return last softmax 'logps' output and hidden state 

    """ 

    def forward(self, input, hidden):
        embed = self.embedding(input)
        lstm_out, hidden = self.lstm(embed, hidden)

        # We just want the last sequence step output which we'll use to classify
        # lstm_out shape is (sequence_steps, batch_size, hidden_units)
        # Output fully-connected layer
        out = self.dropout(self.fc(lstm_out[-1, :, :]))        
        return F.log_softmax(out, dim=1), hidden

    # hidden state 

    """ 
    Initializes hidden state 

        Parameters
        ----------
                - Create two new tensors with sizes n_layers x batch_size x hidden_dim,
                - Initialized to zero, for hidden state and cell state of LSTM
        Returns
        -------
            hidden 
    """
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        return (weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_(),
                  weight.new(self.lstm_layers, batch_size, self.lstm_size).zero_())

### View Model

In [14]:
model = TextClassifier(len(vocab), 10, 6, 5, dropout=0.1, lstm_layers=2)
model.embedding.weight.data.uniform_(-1, 1)
input = torch.randint(0, 1000, (5, 4), dtype=torch.int64)
hidden = model.init_hidden(4)

logps, _ = model.forward(input, hidden)
print(logps)

tensor([[-1.7506, -1.5485, -1.5212, -1.8128, -1.4605],
        [-1.7071, -1.5651, -1.4988, -1.8127, -1.5008],
        [-1.7194, -1.5701, -1.5170, -1.8235, -1.4611],
        [-1.7034, -1.5821, -1.5359, -1.8357, -1.4371]])


## Training
### DataLoaders and Batching
Now we should build a generator that we can use to loop through our data. It'll be more efficient if we can pass our sequences in as batches. Our input tensors should look like `(sequence_length, batch_size)`. So if our sequences are 40 tokens long and we pass in 25 sequences, then we'd have an input size of `(40, 25)`.

If we set our sequence length to 40, what do we do with messages that are more or less than 40 tokens? For messages with fewer than 40 tokens, we will pad the empty spots with zeros. We should be sure to **left** pad so that the RNN starts from nothing before going through the data. If the message has 20 tokens, then the first 20 spots of our 40 long sequence will be 0. If a message has more than 40 tokens, we'll just keep the first 40 tokens.

In [15]:
def dataloader(twit_content, labels, sequence_length=30, batch_size=32, shuffle=True):

    if shuffle:
        indices = list(range(len(twit_content)))
        random.shuffle(indices)
        twit_content = [twit_content[idx] for idx in indices]
        labels = [labels[idx] for idx in indices]

    total_sequences = len(twit_content)

    for ii in range(0, total_sequences, batch_size):
        batch_twit_content = twit_content[ii: ii+batch_size]
        
        batch = torch.zeros((sequence_length, len(batch_twit_content)), dtype=torch.int64)
        for batch_num, tokens in enumerate(batch_twit_content):
            token_tensor = torch.tensor(tokens)
            start_idx = max(sequence_length - len(token_tensor), 0)
            batch[start_idx:, batch_num] = token_tensor[:sequence_length]
        
        label_tensor = torch.tensor(labels[ii: ii+len(batch_twit_content)])
        
        yield batch, label_tensor

### Training and  Validation
With our data in nice shape, we'll split it into training and validation sets.

In [16]:
"""
Split data into training and validation datasets. Use an appropriate split size.
The features are the `token_ids` and the labels are the `sentiments`.
"""   
split_frac = 0.7
split_idx = int(len(token_ids)*split_frac)

train_features, valid_features = token_ids[:split_idx], token_ids[split_idx:]
train_labels, valid_labels = sentiments[:split_idx], sentiments[split_idx:]

In [17]:
text_batch, labels = next(iter(dataloader(train_features, train_labels, sequence_length=20, batch_size=64)))
model = TextClassifier(len(vocab)+1, 200, 128, 5, dropout=0.)
hidden = model.init_hidden(64)
logps, hidden = model.forward(text_batch, hidden)

### Training
It's time to train the neural network!

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = TextClassifier(len(vocab)+1, 1024, 512, 5, lstm_layers=2, dropout=0.2)
model.embedding.weight.data.uniform_(-1, 1)
model.to(device)

TextClassifier(
  (embedding): Embedding(17602, 1024)
  (lstm): LSTM(1024, 512, num_layers=2, dropout=0.2)
  (dropout): Dropout(p=0.2)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)

In [19]:
"""
Train your model with dropout. Make sure to clip your gradients.
Print the training loss, validation loss, and validation accuracy for every 100 steps.
"""

epochs = 3
batch_size = 512
learning_rate = 0.001

clip=5 # gradient clipping

print_every = 100
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model.train()

for epoch in range(epochs):
    print('Starting epoch {}'.format(epoch + 1))
    
    steps = 0
    for text_batch, labels in dataloader(train_features, train_labels, batch_size=batch_size):
        steps += 1
        hidden = model.init_hidden(labels.shape[0])
        
        # Set Device
        text_batch, labels = text_batch.to(device), labels.to(device)
        
        for each in hidden:
            each.to(device)
        
        # TODO Implement: Train Model
        model.zero_grad()
        
        output, hidden = model(text_batch, hidden)     
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels)
        loss.backward()       
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()          
        
        if steps % print_every == 0:
            model.eval()
            
            val_losses = []
            for inputs, labels in dataloader(valid_features, valid_labels, batch_size=batch_size):

                # Creating new variables for the hidden state, otherwise
                # we'd backprop through the entire training history
                #val_h = tuple([each.data for each in val_h])

                inputs, labels = inputs.to(device), labels.to(device)
                
                val_h = model.init_hidden(labels.shape[0])
                for each in val_h:
                    each.to(device)                

                output, val_h = model(inputs, val_h)
                val_loss = criterion(output.squeeze(), labels)

                val_losses.append(val_loss.item())            
            # TODO Implement: Print metrics
            
            model.train()
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(steps),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))

Starting epoch 1
Epoch: 1/3... Step: 100... Loss: 1.201687... Val Loss: 1.177436
Epoch: 1/3... Step: 200... Loss: 1.102004... Val Loss: 1.104870
Epoch: 1/3... Step: 300... Loss: 1.084292... Val Loss: 1.061992
Epoch: 1/3... Step: 400... Loss: 1.039838... Val Loss: 1.030276
Epoch: 1/3... Step: 500... Loss: 1.041137... Val Loss: 1.019772
Epoch: 1/3... Step: 600... Loss: 0.975738... Val Loss: 1.006457
Epoch: 1/3... Step: 700... Loss: 1.031566... Val Loss: 0.990320
Epoch: 1/3... Step: 800... Loss: 0.983875... Val Loss: 0.996749
Epoch: 1/3... Step: 900... Loss: 0.946176... Val Loss: 0.980392
Epoch: 1/3... Step: 1000... Loss: 0.975880... Val Loss: 0.968134
Epoch: 1/3... Step: 1100... Loss: 0.935730... Val Loss: 0.962626
Epoch: 1/3... Step: 1200... Loss: 0.989893... Val Loss: 0.959935
Epoch: 1/3... Step: 1300... Loss: 1.020337... Val Loss: 0.955887
Epoch: 1/3... Step: 1400... Loss: 0.974195... Val Loss: 0.950747
Starting epoch 2
Epoch: 2/3... Step: 100... Loss: 0.893755... Val Loss: 0.950822
E

## Making Predictions
### Prediction 
Okay, now that you have a trained model, try it on some new twits and see if it works appropriately. Remember that for any new text, you'll need to preprocess it first before passing it to the network. Implement the `predict` function to generate the prediction vector from a message.

In [20]:
def predict(text, model, vocab):
    """ 
    Make a prediction on a single sentence.

    Parameters
    ----------
        text : The string to make a prediction on.
        model : The model to use for making the prediction.
        vocab : Dictionary for word to word ids. The key is the word and the value is the word id.

    Returns
    -------
        pred : Prediction vector
    """    
    
    # TODO Implement
    
    tokens = preprocess(text)
    
    tokens = [word for word in tokens if word in filtered_words]
    tokens = [vocab[word] for word in tokens] 
        
    text_input = torch.tensor(tokens).unsqueeze(1)
    
    hidden  = model.init_hidden(text_input.size(1))
    
    logps, _ = model.forward(text_input, hidden)
    pred = torch.exp(logps)
    
    return pred.to('cpu').detach().numpy()

In [21]:
text = "amazon prime perfect breakout strategy"
model.eval()
model.to("cpu")
predict(text, model, vocab)

array([[  9.18790756e-05,   8.21938389e-04,   6.72553084e-04,
          9.79805598e-04,   9.97433841e-01]], dtype=float32)

### Questions: What is the prediction of the model? What is the uncertainty of the prediction?
** TODO: Answer Question**

Now we have a trained model and we can make predictions. We can use this model to track the sentiments of various stocks by predicting the sentiments of twits as they are coming in. Now we have a stream of twits. For each of those twits, pull out the stocks mentioned in them and keep track of the sentiments. Remember that in the twits, ticker symbols are encoded with a dollar sign as the first character, all caps, and 2-4 letters, like $AAPL. Ideally, you'd want to track the sentiments of the stocks in your universe and use this as a signal in your larger model(s).

## Testing
### Load the Data 

In [22]:
with open(os.path.join('..', '..', 'data', 'project_6_stocktwits', 'test_twits.json'), 'r') as f:
    test_data = json.load(f)

### Twit Stream

In [23]:
def twit_stream():
    for twit in test_data['data']:
        yield twit

next(twit_stream())

{'message_body': '$JWN has moved -1.69% on 10-31. Check out the movement and peers at  https://dividendbot.com?s=JWN',
 'timestamp': '2018-11-01T00:00:05Z'}

Using the `prediction` function, let's apply it to a stream of twits.

In [24]:
def score_twits(stream, model, vocab, universe):
    """ 
    Given a stream of twits and a universe of tickers, return sentiment scores for tickers in the universe.
    """
    for twit in stream:

        # Get the message text
        text = twit['message_body']
        symbols = re.findall('\$[A-Z]{2,4}', text)
        score = predict(text, model, vocab)

        for symbol in symbols:
            if symbol in universe:
                yield {'symbol': symbol, 'score': score, 'timestamp': twit['timestamp']}

In [25]:
universe = {'$BBRY', '$AAPL', '$AMZN', '$BABA', '$YHOO', '$LQMT', '$FB', '$GOOG', '$BBBY', '$JNUG', '$SBUX', '$MU'}
score_stream = score_twits(twit_stream(), model, vocab, universe)

next(score_stream)

{'symbol': '$AAPL',
 'score': array([[ 0.44229251,  0.04212086,  0.08983315,  0.02372071,  0.40203276]], dtype=float32),
 'timestamp': '2018-11-01T00:00:18Z'}

That's it. You have successfully built a model for sentiment analysis! 

## Submission
Now that you're done with the project, it's time to submit it. Click the submit button in the bottom right. One of our reviewers will give you feedback on your project with a pass or not passed grade. You can continue to the next section while you wait for feedback.